In [25]:
import os
import pandas as pd
import numpy as np
from scipy import stats

path = os.path.join(os.getenv("HOME"),'Dropbox/Research')

def derive_coef (x,y,xerr,yerr,N):
    rho_sim = []
    p_sim = []
    for i in range(N):
        #Generate random weights using mean weights with measurement error
        y_i = np.random.normal(y,yerr)
        x_i = np.random.normal(x,xerr)
        #Calculate correlation coefficient and p-value
        rho_i,p_i = stats.spearmanr(x_i,y_i)
        rho_sim.append(rho_i)
        p_sim.append(p_i)
    #====여기까지가 MC임
    rho = np.median(rho_sim)
    err_rho = np.mean([np.percentile(rho_sim,84.135) - rho,
                      rho-np.percentile(rho_sim,15.865)])
    rho,err_rho = round(rho,4), round(err_rho,4)
    
    p = np.median(p_sim)
    err_p = np.mean([np.percentile(p_sim,84.135) - p,
                     p-np.percentile(p_sim,15.865)])
    
    return (rho, err_rho, p, err_p)

df = pd.read_csv(os.path.join(path,'Hydrated_asteroid','Manuscript','Archive_compile.csv'))

## #Deriving the Correlation Coeifficients

In [38]:
df_Coef = pd.DataFrame({})    
N = 10000

#Polarimetry
for polarimetry in ['Pmin','h','a0','amin']:
    df_new = dict()
    df_new['COMP1']=[polarimetry]
    for others in ['R_UV','R_07','R_27','OE','b','Gamma']:
        df_ = df.dropna(subset=[others])
        df_ = df_[df_['R_07_flag']!=-1]
        df_ = df_.dropna(subset=['Pmin','h','a0'])
        if polarimetry == 'amin':
            df_ = df_.dropna(subset=['amin'])
        if others in ['OE','b']:
            eother = [1e-10]*len(df_[others])
        else:
            df_ = df_.dropna(subset=['e'+others])
            eother = df_['e'+others]
            
            
        rho,erho,_,_ = derive_coef(abs(df_[polarimetry]),
                                   df_[others],
                                   abs(df_['e'+polarimetry]),
                                   eother,
                                   N)
        df_new['COMP2']=[others]
        df_new['rho']=[rho]
        df_new['erho']=[erho]
        df_new['sample number']=[len(df_)]
        df_new['N_iter']=[N]
        df_Coef = pd.concat([df_Coef,pd.DataFrame(df_new)])
        print(df_new)
        
for photometry in ['OE','b']:
    df_new = dict()
    df_new['COMP1']=[photometry]
    for spectroscopy in ['R_UV','R_07','R_27','Gamma']:
        df_ = df.dropna(subset=[photometry])
        df_ = df_.dropna(subset=[spectroscopy])
        rho,erho,_,_ = derive_coef(df_[photometry],
                                   df_[spectroscopy],
                                   [1e-10]*len(df_[photometry]),
                                   df_['e'+spectroscopy],
                                   N)
        df_new['COMP2']=[spectroscopy]
        df_new['rho']=[rho]
        df_new['erho']=[erho]
        df_new['sample number']=[len(df_)]
        df_new['N_iter']=[N]
        df_Coef = pd.concat([df_Coef,pd.DataFrame(df_new)])
        
#Polarimetry
for observation in ['Pmin','h','a0','amin','R_UV','R_07','R_27','OE','b','Gamma']:
    df_new = dict()
    df_new['COMP1']=['pV_s']
    df_ = df.dropna(subset=[observation])
    df_ = df_.dropna(subset=['pV_s'])
    if observation in ['OE','b']:
        eobservation = [1e-10]*len(df_)
    else:
        df_ = df_.dropna(subset=['e'+observation])
        eobservation = df_['e'+observation]
    rho,erho,_,_ = derive_coef(df_['pV_s'],
                               abs(df_[observation]),
                               df_['epV_s'],
                               eobservation,
                               N)
    df_new['COMP2']=[observation]
    df_new['rho']=[rho]
    df_new['erho']=[erho]
    df_new['sample number']=[len(df_)]
    df_new['N_iter']=[N]
    df_Coef = pd.concat([df_Coef,pd.DataFrame(df_new)])
Filename = os.path.join(path,'Hydrated_asteroid','Manuscript','Archive_Coeiff_compile.csv')
df_Coef.to_csv(Filename,index=False)

{'COMP1': ['Pmin'], 'COMP2': ['R_UV'], 'rho': [-0.6709], 'erho': [0.0329], 'sample number': [49], 'N_iter': [10000]}
{'COMP1': ['Pmin'], 'COMP2': ['R_07'], 'rho': [-0.5762], 'erho': [0.0439], 'sample number': [69], 'N_iter': [10000]}
{'COMP1': ['Pmin'], 'COMP2': ['R_27'], 'rho': [-0.5386], 'erho': [0.1395], 'sample number': [19], 'N_iter': [10000]}
{'COMP1': ['Pmin'], 'COMP2': ['OE'], 'rho': [0.8061], 'erho': [0.0727], 'sample number': [10], 'N_iter': [10000]}
{'COMP1': ['Pmin'], 'COMP2': ['b'], 'rho': [-0.697], 'erho': [0.0606], 'sample number': [10], 'N_iter': [10000]}
{'COMP1': ['Pmin'], 'COMP2': ['Gamma'], 'rho': [-0.3571], 'erho': [0.2857], 'sample number': [8], 'N_iter': [10000]}
{'COMP1': ['h'], 'COMP2': ['R_UV'], 'rho': [-0.4324], 'erho': [0.0527], 'sample number': [49], 'N_iter': [10000]}
{'COMP1': ['h'], 'COMP2': ['R_07'], 'rho': [-0.55], 'erho': [0.0496], 'sample number': [69], 'N_iter': [10000]}
{'COMP1': ['h'], 'COMP2': ['R_27'], 'rho': [-0.4193], 'erho': [0.1342], 'sample